In [1]:
import cv2
import mediapipe as mp
import time

class PoseDetector():
    def __init__(self,mode =False,smooth=True,detectionCon=0.5,trackingCon=0.5,model_complexity=1):
        self.mode=mode
        self.smooth=smooth
        self.detectionCon=detectionCon
        self.trackingCon=trackingCon
        self.model_complexity=model_complexity
        self.mpPose=mp.solutions.pose
        self.mpDraw=mp.solutions.drawing_utils
        self.pose=self.mpPose.Pose(
        static_image_mode=False,
        model_complexity=1,
        smooth_landmarks=True,
        enable_segmentation=False,
        smooth_segmentation=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
        )
    def findPose(self,img,draw=True):
        imgRBG=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        self.results=self.pose.process(imgRBG)
        if self.results.pose_landmarks:
            if draw:
                self.mpDraw.draw_landmarks(img,self.results.pose_landmarks,self.mpPose.POSE_CONNECTIONS)
        return img
    def findPosition(self,img,draw=True):
        lmList=[]
        if self.results.pose_landmarks:
            for id,lm in enumerate(self.results.pose_landmarks.landmark):
                h,w,c=img.shape
                lmList.append([id,lm.x*w,lm.y*h])
                if draw:
                    cv2.circle(img,(int(lm.x*w),int(lm.y*h)),5,(255,0,0),cv2.FILLED)
        return lmList
        
    


def main():
    cap=cv2.VideoCapture(0)
    def rescale_frame(frame, percent=75):
        width = int(frame.shape[1] * percent/ 100)
        height = int(frame.shape[0] * percent/ 100)
        dim = (width, height)
        return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)
    iTime=0
    pTime=0
    detector=PoseDetector()
    while True:
        success,img=cap.read()
        
        img=detector.findPose(img)
        lmList=detector.findPosition(img)
        # print(lmList)
        
        iTime=time.time()
        fps=1/(iTime-pTime)
        cv2.putText(img = img,text = str(int(fps)),org = (70,50),fontFace = cv2.FONT_HERSHEY_PLAIN, fontScale = 3.0,color = (125, 246, 55),thickness = 3)
        pTime=iTime
        cv2.imshow("img",img)
        if cv2.waitKey(1) & 0xFF==ord('a'):
            break
    cap.release()
    cv2.destroyAllWindows()
if __name__=="__main__":
    main()